In [1]:
# + tags=["parameters"]
# declare a list tasks whose products you want to use as inputs
upstream = ['vectorizer_countVec','topic_modeling_disaster_type_final']
random_seed = 42
from tools_rjg import *
import pandas as pd
import re, nltk, spacy, gensim
nltk.download('punkt')
import pyLDAvis.sklearn
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gillrobe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
# Parameters
upstream = {
    "vectorizer_countVec": {
        "nb": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\vectorizer_countVec.ipynb",
        "vectorizer": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\vectorizer_countVec.pkl",
    },
    "topic_modeling_disaster_type_final": {
        "nb": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\topic_modeling_disaster_type_final.ipynb",
        "lda_model_earthquake": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_model_earthquake.pkl",
        "lda_model_fire": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_model_fire.pkl",
        "lda_model_flood": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_model_flood.pkl",
        "lda_model_hurricane": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_model_hurricane.pkl",
        "lda_topics_disaster_type": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\lda_topics_disaster_type.csv",
    },
}
product = {
    "nb": "C:\\Users\\gillrobe\\DataScience\\umads_697_data_medics\\pipeline\\output\\topic_modeling_disaster_type_results.ipynb"
}


In [3]:
df_all = generate_disaster_type_dataframe(disaster_types = ('earthquake', 'fire', 'flood', 'hurricane')
                                          , dev_train_test= ('dev', 'train', 'test'))

In [4]:
with open("output\\vectorizer_countVec.pkl", "rb") as f:
    vectorizer = pickle.load(f)

In [5]:
with open("output\\lda_model_earthquake.pkl", "rb") as f:
    lda_model_earthquake = pickle.load(f)
with open("output\\lda_model_fire.pkl", "rb") as f:
    lda_model_fire = pickle.load(f)
with open("output\\lda_model_flood.pkl", "rb") as f:
    lda_model_flood = pickle.load(f)
with open("output\\lda_model_hurricane.pkl", "rb") as f:
    lda_model_hurricane = pickle.load(f)

In [6]:
df_all['tweet_text_cleaned'] = df_all['tweet_text'].apply(lambda x: tweet_preprocessing(x))
df_all['lemmatized'] = df_all['tweet_text_cleaned'].apply(lambda x: lemmatize_tweet_text(x, allowed_postags=('NOUN', 'ADJ', 'VERB', 'ADV')))

In [7]:
df_all.sample(100)

,tweet_id,tweet_text,class_label,disaster_type,tweet_text_cleaned,lemmatized
28778,1168281553413332992,Ibuprofen flies off the shelves faster than wa...,not_humanitarian,hurricane,"[ibuprofen, flies, shelves, faster, than, water]",ibuprofen fly shelf fast water
3038,901800385194708998,"Yep, this looks frighteningly similar to TS Al...",sympathy_and_support,hurricane,"[this, looks, frighteningly, similar, allison,...",look frighteningly similar allison stay safe f...
1895,903059974040895488,MY WORK IS DOING A DONATION DRIVE FOR HURRICAN...,rescue_volunteering_or_donation_effort,hurricane,"[work, doing, donation, drive, hurricane, harv...",work do donation drive hurricane harvey want d...
4247,914710368798220288,Coalition Of Minnesota Puerto Ricans Helping H...,rescue_volunteering_or_donation_effort,hurricane,"[coalition, minnesota, puerto, ricans, helping...",coalition rican help hurricane maria victim cl...
8070,902654149464137729,Hurricane Harvey brings devastating floods to ...,other_relevant_information,hurricane,"[hurricane, harvey, brings, devastating, flood...",harvey bring devastating flood market economy
...,...,...,...,...,...,...
520,1042012665932070913,RT @BlueCrossNC: Find open pharmacies and ways...,caution_and_advice,hurricane,"[find, open, pharmacies, ways, access, care, a...",find open pharmacy way access care area impact...
3379,1107920827369091072,Death toll from the effects of #CycloneIdai ha...,injured_or_dead_people,hurricane,"[death, toll, from, effects, cyclone, idai, ri...",death toll effect cyclone rise many still miss...
1039,721645642573946880,Strongest earthquake to hit Ecuador in decades...,injured_or_dead_people,earthquake,"[strongest, earthquake, ecuador, decades, kill...",strong earthquake ecuador decade kill least sa...
7004,1001819952066768903,@JRubinBlogger And how is Trump dealing with d...,not_humanitarian,flood,"[trump, dealing, with, destructive, hawaiian, ...",trump deal destructive hawaiian volcanic activ...


In [8]:
earthquake_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='earthquake']['lemmatized']))
fire_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='fire']['lemmatized']))
flood_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='flood']['lemmatized']))
hurricane_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='hurricane']['lemmatized']))

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['donate'] not in stop_words.
  warnings.warn(


## Performance Stats

In [9]:
# Log Likelyhood: Higher the better
print("Log Likelihood Earthquake: ", lda_model_earthquake.score(earthquake_text_vectorized))
print("Log Likelihood Fire: ", lda_model_fire.score(fire_text_vectorized))
print("Log Likelihood Flood: ", lda_model_flood.score(flood_text_vectorized))
print("Log Likelihood Hurricane: ", lda_model_hurricane.score(hurricane_text_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity Earthquake: ", lda_model_earthquake.perplexity(earthquake_text_vectorized))
print("Perplexity Fire: ", lda_model_fire.perplexity(fire_text_vectorized))
print("Perplexity Flood: ", lda_model_flood.perplexity(flood_text_vectorized))
print("Perplexity Hurricane: ", lda_model_hurricane.perplexity(hurricane_text_vectorized))

Log Likelihood Earthquake:  -489957.40154870134


Log Likelihood Fire:  -849270.4203478852


Log Likelihood Flood:  -1017564.9037416613


Log Likelihood Hurricane:  -3202283.332471243


Perplexity Earthquake:  1545.0721822945434


Perplexity Fire:  1653.9649026362408


Perplexity Flood:  1798.3129541093047


Perplexity Hurricane:  1895.7263098863632


In [10]:
get_dominant_topics(lda_model_earthquake,earthquake_text_vectorized)

,Topic0,Topic1,Topic2,dominant_topic
Doc0,0.290000,0.030000,0.680000,2
Doc1,0.870000,0.070000,0.070000,0
Doc2,0.030000,0.640000,0.330000,1
Doc3,0.350000,0.410000,0.250000,1
Doc4,0.060000,0.060000,0.880000,2
Doc5,0.480000,0.050000,0.480000,0
Doc6,0.040000,0.930000,0.040000,1
Doc7,0.060000,0.060000,0.890000,2
Doc8,0.790000,0.090000,0.120000,0
Doc9,0.830000,0.080000,0.080000,0


In [11]:
get_dominant_topics(lda_model_fire,fire_text_vectorized)

,Topic0,Topic1,Topic2,dominant_topic
Doc0,0.170000,0.670000,0.170000,1
Doc1,0.030000,0.030000,0.940000,2
Doc2,0.050000,0.900000,0.050000,1
Doc3,0.840000,0.070000,0.100000,0
Doc4,0.060000,0.360000,0.580000,2
Doc5,0.040000,0.920000,0.040000,1
Doc6,0.040000,0.920000,0.040000,1
Doc7,0.040000,0.040000,0.920000,2
Doc8,0.090000,0.550000,0.370000,1
Doc9,0.190000,0.620000,0.190000,1


In [12]:
get_dominant_topics(lda_model_flood,flood_text_vectorized)

,Topic0,Topic1,Topic2,dominant_topic
Doc0,0.180000,0.770000,0.050000,1
Doc1,0.050000,0.780000,0.170000,1
Doc2,0.380000,0.540000,0.080000,1
Doc3,0.060000,0.890000,0.060000,1
Doc4,0.090000,0.770000,0.140000,1
Doc5,0.020000,0.300000,0.680000,2
Doc6,0.030000,0.940000,0.030000,1
Doc7,0.870000,0.110000,0.020000,0
Doc8,0.860000,0.030000,0.120000,0
Doc9,0.030000,0.940000,0.030000,1


In [13]:
get_dominant_topics(lda_model_hurricane, hurricane_text_vectorized)

,Topic0,Topic1,Topic2,dominant_topic
Doc0,0.280000,0.040000,0.680000,2
Doc1,0.030000,0.140000,0.830000,2
Doc2,0.030000,0.950000,0.030000,1
Doc3,0.150000,0.260000,0.590000,2
Doc4,0.950000,0.030000,0.030000,0
Doc5,0.250000,0.020000,0.730000,2
Doc6,0.770000,0.110000,0.120000,0
Doc7,0.040000,0.130000,0.830000,2
Doc8,0.020000,0.360000,0.620000,2
Doc9,0.340000,0.110000,0.550000,2


## Get Topic Distribution

In [14]:
get_topic_distribution(lda_model_earthquake, vectorizer)

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aameen,abaco,abandon,abandon pet,abfire,ability,able,able donate,able help,abortion,...,yike,yorker,young,young child,youth,yymfire,zealand,zimbabwean,zone,zone evacuate
Topic0,16.072482,0.333333,0.334072,0.333333,0.333333,1.842590,0.346033,0.333663,0.357965,0.333333,...,0.567831,0.335351,0.435039,0.334200,0.360916,0.333333,26.224837,0.333333,8.709885,2.241581
Topic1,0.345286,0.333333,0.334197,0.333333,0.333333,0.656331,26.699035,1.201489,7.057862,0.333333,...,4.080497,1.096546,12.545070,1.128855,0.375878,0.333333,0.353850,0.333333,0.478382,0.341070
Topic2,0.389071,0.333333,0.777419,0.333333,0.333333,0.515900,0.447955,0.340471,0.364838,0.333333,...,0.668461,0.335478,10.068848,0.646290,2.476559,0.333333,11.801879,0.333333,14.407986,0.336123


In [15]:
get_topic_distribution(lda_model_fire, vectorizer)

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aameen,abaco,abandon,abandon pet,abfire,ability,able,able donate,able help,abortion,...,yike,yorker,young,young child,youth,yymfire,zealand,zimbabwean,zone,zone evacuate
Topic0,0.333333,0.333333,2.774917,0.335123,0.356967,0.391713,0.372541,0.363156,0.333596,0.334386,...,0.333333,0.333333,0.419828,2.651837,0.335506,0.369127,0.333333,0.333333,43.861692,0.333333
Topic1,0.333333,0.333333,18.661338,3.267983,14.082438,0.350413,41.949764,4.782497,2.036028,0.419086,...,0.333333,0.333333,17.153513,6.546918,5.276854,11.554626,0.333333,0.333333,0.376290,0.333333
Topic2,0.333333,0.333333,0.842674,0.334522,0.352818,3.880096,0.502520,0.339388,0.337750,1.661617,...,0.333333,0.333333,17.795753,0.370851,0.334915,0.386900,0.333333,0.333333,1.014292,0.333333


In [16]:
get_topic_distribution(lda_model_flood, vectorizer)

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aameen,abaco,abandon,abandon pet,abfire,ability,able,able donate,able help,abortion,...,yike,yorker,young,young child,youth,yymfire,zealand,zimbabwean,zone,zone evacuate
Topic0,0.333806,0.333333,5.008241,0.688236,0.333333,0.509957,34.128366,0.360233,8.167390,0.333333,...,0.333333,0.362379,0.506311,3.630370,0.480628,0.333333,0.333333,0.333333,0.642682,0.333333
Topic1,0.333694,0.333333,0.380639,0.372659,0.333333,0.438830,19.179508,0.769730,0.359055,0.333333,...,0.333333,0.375276,56.140387,0.454526,25.490031,0.333333,0.333333,0.333333,7.854235,0.333333
Topic2,0.755636,0.333333,0.519816,0.333806,0.333333,5.827860,17.901032,0.334792,0.365608,0.333333,...,0.333333,1.231153,0.441722,0.396678,0.354652,0.333333,0.333333,0.333333,8.269187,0.333333


In [17]:
get_topic_distribution(lda_model_hurricane, vectorizer)

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aameen,abaco,abandon,abandon pet,abfire,ability,able,able donate,able help,abortion,...,yike,yorker,young,young child,youth,yymfire,zealand,zimbabwean,zone,zone evacuate
Topic0,0.333333,68.287290,0.353920,0.335201,0.333333,0.389125,0.415523,0.335488,0.337602,0.341905,...,18.966773,0.338994,0.355101,0.345728,0.353052,0.333333,0.333333,0.402321,367.382061,10.324387
Topic1,0.333333,0.358130,0.342719,0.355038,0.333333,0.346010,26.869073,3.754118,0.462674,11.023840,...,0.338435,10.226409,0.842959,0.623532,26.995064,0.333333,0.333333,120.297859,0.379947,0.350678
Topic2,0.333333,0.388771,35.217037,15.900986,0.333333,18.521446,153.612300,0.338921,20.347976,0.508421,...,0.465499,0.335475,60.644338,3.833090,0.380537,0.333333,0.333333,0.432519,0.535766,0.410759


## Show Top N Topics

In [18]:
show_topics(vectorizer, fitted_lda_model=lda_model_earthquake, n_words=20)

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
_Topic 0,prayer,safe,injure,thought,affect,kill,magnitude,dead,pray,family,stay,least,hope,thought prayer,keep,stay safe,victim,quake,send,news
_Topic 1,help,donate,death,victim,rescue,toll,death toll,relief,need,rise,team,effort,support,say,thank,donation,toll rise,quake,survivor,need help
_Topic 2,pray,damage,aftershock,building,area,report,road,collapse,still,city,come,know,many,good,last,feel,make,cause,affected,need


In [19]:
show_topics(vectorizer, fitted_lda_model=lda_model_fire, n_words=20)

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
_Topic 0,death,toll,death toll,camp,dead,miss,kill,deadly,least,rise,news,toll rise,still,history,state,continue,search,break,official,find
_Topic 1,help,victim,donate,camp,relief,need,support,donation,effort,affect,evacuation,fund,thank,evacuee,money,raise,animal,good,community,make
_Topic 2,home,lose,destroy,burn,many,firefighter,pray,life,year,climate,cause,time,state,change,area,know,go,blame,live,family


In [20]:
show_topics(vectorizer, fitted_lda_model=lda_model_flood, n_words=20)

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
_Topic 0,help,need,relief,rescue,food,camp,family,affect,pray,stand,operation,rebuild,life,relief camp,team,need help,item,save,contact,friend
_Topic 1,relief,donate,victim,fund,help,relief fund,donation,affect,work,material,support,thank,effort,send,relief material,contribute,donate relief,money,come,medical
_Topic 2,flooding,state,damage,disaster,year,rain,say,flash,miss,news,assistance,bad,prayer,time,death,thought,take,cause,national,go


In [21]:
show_topics(vectorizer, fitted_lda_model=lda_model_hurricane, n_words=20)

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Word 15,Word 16,Word 17,Word 18,Word 19
_Topic 0,damage,storm,evacuation,destroy,wind,flooding,death,cause,news,area,island,update,coast,state,home,category,destruction,report,toll,rain
_Topic 1,help,relief,cyclone,donate,affect,victim,prayer,effort,thought,donation,support,team,send,thought prayer,disaster,volunteer,rescue,provide,relief effort,thank
_Topic 2,need,help,food,pray,shelter,safe,family,take,time,go,home,stay,good,know,emergency,supply,many,hope,need help,prepare


## Extract most representative sentence for each topic

In [22]:
pd.options.display.max_colwidth = 1000

In [23]:
earthquake_top_sentence = extract_most_representative_sentence(lda_model_earthquake, earthquake_text_vectorized, df_all[df_all['disaster_type']=='earthquake'])
for topic in range(lda_model_earthquake.n_components):
    print('----------------------')
    print(f'Topic_{topic}', earthquake_top_sentence[f'Topic_{topic}'][1]['tweet_text'])

----------------------
Topic_0 5031    My heart goes out to all the people who lost their lives in todays #earthquake. My thoughts and prayers are with their families. May all the departed souls rest in peace and May Allah protect all of us from these natural calamities
Name: tweet_text, dtype: object
----------------------
Topic_1 5162    Download to watch LIVE:  NDMA, Pak Army playing vital role in quake rescue, relief efforts: SAPM Awan #Pak #Live #NEWS #Channel #ARYNewsLiveHD #Pakistan #WorldNews #OZOOTV #Android
Name: tweet_text, dtype: object
----------------------
Topic_2 6216    2/4 Three communication bridges near Jatlan damaged. Road Jatlan-Mangla settled at multiple locations rendering it unusable. Medium to minor damages to weak / old structures in Mirpur, Jatlan and Jarikas. No damage to Mangla Dam reported. #earthquake
Name: tweet_text, dtype: object


In [24]:
fire_top_sentence = extract_most_representative_sentence(lda_model_fire, fire_text_vectorized, df_all[df_all['disaster_type']=='fire'])
for topic in range(lda_model_fire.n_components):
    print('----------------------')
    print(f'Topic_{topic}', fire_top_sentence[f'Topic_{topic}'][1]['tweet_text'])

----------------------
Topic_0 639    The #CampFire still raging in Northern California has become the deadliest wildfire in state history as authorities locate the remains of 13 more people, bringing the death toll to at least 42. Officials say the fire has scorched more than 117,000 acres.
Name: tweet_text, dtype: object
----------------------
Topic_1 7227    Its brew day for #ResilienceIPA, brewed to support the community of Butte County who were impacted by the Camp Fire. 100% of Resilience sales will be donated to the @SierraNevada Camp Fire Relief Fund to help rebuild its community. #ButteStrong #SierraNevada. ἷB❤️
Name: tweet_text, dtype: object
----------------------
Topic_2 239    Noone in Greece is without a heavy heart today. Human lives were lost, animals burnt, houses &amp; lifetime savings gone to ashes. Thank you all friends &amp; colleagues for your thoughts and nice words. They mean so much here, what with the black skies and smell and sadness. #wildfires
Name: tweet_t

In [25]:
flood_top_sentence = extract_most_representative_sentence(lda_model_flood, flood_text_vectorized, df_all[df_all['disaster_type']=='flood'])
for topic in range(lda_model_flood.n_components):
    print('----------------------')
    print(f'Topic_{topic}', flood_top_sentence[f'Topic_{topic}'][1]['tweet_text'])

----------------------
Topic_0 1131    #KeralaFloods #KeralaFloodRelief ὄ9Indian Coast Guards Rescue &amp; Relief helicopter carried 1000 kg of general medicines,food packets,drinking water,sanitary pads,baby diapers,bed sheets toothpaste etc..&amp; dropped them remotest &amp; inaccessible flooded areas
Name: tweet_text, dtype: object
----------------------
Topic_1 3888    The Faculty, Residents &amp; Staff @JipmerNephro contributed Rice, Sugar &amp; Pulses worth INR 21000/- for #KeralaFloodRelief - in addition to their personal contributions, volunteering for relief work &amp; joining #JIPMER fund collection drive from staff salary contributions
Name: tweet_text, dtype: object
----------------------
Topic_2 1873    Nebraska Severe Weather Awareness Week continues today with the topic of Tornadoes. Do you know what to do if a Tornado Warning is issued for your area or you see a tornado? NOW is the time to make a plan! Use the TEST Tornado Warning today to practice your plan. #newx #swa

In [26]:
hurricane_top_sentence = extract_most_representative_sentence(lda_model_hurricane, hurricane_text_vectorized, df_all[df_all['disaster_type']=='hurricane'])
for topic in range(lda_model_hurricane.n_components):
    print('----------------------')
    print(f'Topic_{topic}', hurricane_top_sentence[f'Topic_{topic}'][1]['tweet_text'])

----------------------
Topic_0 29690    Current projections have reduced the likelihood of tropical storm force winds &amp; heavy rain in our area, though high speed winds &amp; gusts are still possible beginning Tuesday. Weather forecasts can change readily &amp; residents are encouraged to monitor the storm &amp; stay tuned.
Name: tweet_text, dtype: object
----------------------
Topic_1 6929    Reach out &amp; help those affected by #cycloneidai Visit any Econet shop &amp; drop your donations or use the EcoCash biller code 320041 Lets come together and help those in need! We thank you for your support! Rt for awareness
Name: tweet_text, dtype: object
----------------------
Topic_2 947    Stay safe out there! Make sure you and your family are prepared with the necessary survival tools as well as insurance! We are here to help keep you and your family safe as well as your belongings! Call today. #StracarInsurance #HurricaneFlorence
Name: tweet_text, dtype: object


In [27]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_earthquake, earthquake_text_vectorized, vectorizer, mds='tsne')
panel

C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\manifold\_t_sne.py:795: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\gillrobe\AppData\Local\pypoetry\Cache\virtualenvs\data-medics-capstone-E3RTzdp6-py3.9\lib\site-packages\sklearn\manifold\_t_sne.py:805: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
0     -1015.700562  1338.981201       1        1  35.618173
1      -387.996002  -373.316162       2        1  33.502264
2     -2184.745605   -60.565800       3        1  30.879563, topic_info=           Term         Freq        Total Category  logprob  loglift
3840       help  1028.000000  1028.000000  Default  30.0000  30.0000
6252     prayer   679.000000   679.000000  Default  29.0000  29.0000
1777     damage   535.000000   535.000000  Default  28.0000  28.0000
7163       safe   567.000000   567.000000  Default  27.0000  27.0000
2328     donate   530.000000   530.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
4173       hour    89.732017   109.143585   Topic3  -5.4364   0.9792
1217       city   140.952982   315.077655   Topic3  -4.9848   0.3707
2645  emergency    99.097330   152.810769   Topic3  -5.3372   0.7420
5559       need   101.081832   401.880152   Topic3  -5.3173  -0.2051
8163       take    84.358600   148.334074   Topic3  -5.4982   0.6107

[151 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
103       1  0.745605    affect
103       2  0.150069    affect
103       3  0.105838    affect
164       1  0.009596  affected
164       3  0.988438  affected
...     ...       ...       ...
9053      2  0.845900      want
9053      3  0.152072      want
9254      2  0.786501      work
9254      3  0.207926      work
9281      3  0.994704     world

[146 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])